# Maps of RMSSSH

In [13]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(196)
cluster

from dask.distributed import Client
client = Client(cluster)
client



Client Scheduler: tcp://172.30.100.1:46821 Dashboard: http://172.30.100.1:34971/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [14]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8182198     hsw24 make_pro albert7a CG      27:14      1 n2642
           8182199     hsw24 make_pro albert7a CG      27:14      1 n2650
           8182193     hsw24 make_pro albert7a CG      28:16      1 n2764
           8182584     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182585     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182586     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182587     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182588     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182589     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182590     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8182351     hsw24 make_pro albert7a  R       1:47      1 n2167
           8182350     hsw24 make_pro albert7a  R       2:31      

## 0. Modules and libraries

In [15]:
import warnings
warnings.filterwarnings("ignore")
import dask 
import numpy as np
import xarray as xr
import time
import numpy.ma as ma
import matplotlib.cm as mplcm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import os 
import time 
import glob

import zarr

%matplotlib inline


## 1. Data 

### AVISO

In [16]:
files_aviso = !ls /scratch/cnt0024/hmg2840/albert7a/AVISO/dt_global_allsat_phy_l4_20*20190101.nc
aviso_local = xr.open_mfdataset(files_aviso,chunks={'latitude':700,'longitude':1000})


In [17]:
aviso_year = aviso_local.sel(time=slice('2009-06-30','2010-10-29'))
myadt = aviso_year.adt
rms_ssh = myadt.std(dim='time')
aviso_mssh = myadt.mean(dim='time') - myadt.mean()


In [18]:
std_dataset=rms_ssh.to_dataset(name='std_sossheig')
std_dataset['std_sossheig'].attrs=myadt.attrs
std_dataset['std_sossheig'].attrs['standard_name']='standart deviation of ssh'
std_dataset['std_sossheig'].attrs['long_name']='standart deviation of sea surface height'
std_dataset.attrs['global_attribute']= 'standart deviation of ssh computed on occigen '
std_dataset.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/AVISO/AVISO_y2009m06d30-y2010m10d29_std-ssh.nc',mode='w')

In [19]:
mean_ssh_eNATL60=aviso_mssh.to_dataset(name='mean_sossheig')
mean_ssh_eNATL60['mean_sossheig'].attrs=myadt.attrs
mean_ssh_eNATL60['mean_sossheig'].attrs['standard_name']='temporal mean of ssh'
mean_ssh_eNATL60['mean_sossheig'].attrs['long_name']='temporal mean of sea surface height'
mean_ssh_eNATL60.attrs['global_attribute']= 'temporal mean of ssh computed on occigen '
mean_ssh_eNATL60.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/AVISO/AVISO_y2009m06d30-y2010m10d29_mean-ssh.nc',mode='w')